In [189]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import re
from datetime import datetime
import os
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [190]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

## 변수 설정

In [191]:
query = input("검색어를 입력하세요 : ")
news_num = int(input("크롤링할 기사 개수를 입력하세요 : "))

In [192]:
print(query, ',' ,news_num)

코로나 , 5


## 페이지 읽어오기

In [193]:
base_url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={query}'

In [194]:
news_dict = {}
idx = 0
page = 1
tmp = {}

In [195]:
print('기사 수집 중입니다.')
while idx < news_num :   
    
    response = requests.get(base_url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    news_list = soup.select('div.news_area')
    next_page = soup.select('div.sc_page_inner > a.btn')

    for news in news_list :
        if news.select('a.info')[-1].text == '네이버뉴스' :
            link = news.select('a.info')[-1]['href']
            driver = webdriver.Chrome('chromedriver.exe')
            driver.get(link)

            while True :
                time.sleep(1.5)
                try : 
                    contents = []
                    driver.find_element(By.XPATH, './/*[@id="cbox_module"]/div[2]/div[9]/a/span[1]').click()
                    html2 = driver.page_source
                    soup2 = BeautifulSoup(html2, 'lxml')

                    title = news.select_one('a.news_tit').text
                    text = news.select_one('a.api_txt_lines.dsc_txt_wrap').text
                    comment_list = soup2.find_all("span", {'class' : "u_cbox_contents"})

                    for comment in comment_list :
                        comment = comment.text
                        contents.append(comment)

                    news_dict[idx] = {'제목' : title, '링크' : link, '본문 요약' : text, '댓글' : contents}
                    idx += 1

                except :
                    break
                
        else :
            tmp[idx] = {'제목' : title, '링크' : link, '본문 요약' : text}

    page += 1
    
    for p in next_page :
        if p.text == str(page) :
            news_url = f'https://search.naver.com/search.naver' + p['href']

print('크롤링이 완료되었습니다.')

기사 수집 중입니다.
크롤링이 완료되었습니다.


In [196]:
df = pd.DataFrame(news_dict).T
df

,제목,링크,본문 요약,댓글
0,곧 5월인데 독감·감기 동시 유행…코로나 환자도 늘어,https://n.news.naver.com/mnews/article/277/000...,특히 코로나19 시대 마스크 착용으로 각종 호흡기 바이러스에 대한 면역이 떨어진 영...,[]
1,봄철 독감 유행 · 감기 환자도↑…코로나19 방역 완화 영향,https://n.news.naver.com/mnews/article/055/000...,"감기 환자도 계속 늘고 있는데, 방역 당국은 코로나19 방역 조치가 완화된 영향으로...",[]
2,눈병인 줄 알았더니 코로나라고?…새 변이가 무서운 이유,https://n.news.naver.com/mnews/article/015/000...,기존 코로나19 오미크론 하위 변이보다 전염성이 강한 새로운 오미크론 하위 변이가 ...,[]
3,눈병인 줄 알았더니 코로나라고?…새 변이가 무서운 이유,https://n.news.naver.com/mnews/article/015/000...,기존 코로나19 오미크론 하위 변이보다 전염성이 강한 새로운 오미크론 하위 변이가 ...,"[목에 칼이 들어와도 백신은 사양합니다~~, 요즘도 코로나로 겁주는 기자가 존재하는..."
4,눈병인 줄 알았더니 코로나라고?…새 변이가 무서운 이유,https://n.news.naver.com/mnews/article/015/000...,기존 코로나19 오미크론 하위 변이보다 전염성이 강한 새로운 오미크론 하위 변이가 ...,"[목에 칼이 들어와도 백신은 사양합니다~~, 요즘도 코로나로 겁주는 기자가 존재하는..."
...,...,...,...,...
58,"복통 치료 쓰던 한약재 ‘초두구’, 코로나19 치료 효과 확인",https://n.news.naver.com/mnews/article/366/000...,"권선오 한의학연 책임연구원 연구팀 한의학에서 가슴이나 배 통증, 구토와 같은 증상을...","[예전엔 지식인들만 글을 썼는데, 이젠 하층민들이 자신이 지식인인 줄 알고 글을 쓴..."
59,"복통 치료 쓰던 한약재 ‘초두구’, 코로나19 치료 효과 확인",https://n.news.naver.com/mnews/article/366/000...,"권선오 한의학연 책임연구원 연구팀 한의학에서 가슴이나 배 통증, 구토와 같은 증상을...","[예전엔 지식인들만 글을 썼는데, 이젠 하층민들이 자신이 지식인인 줄 알고 글을 쓴..."
60,"복통 치료 쓰던 한약재 ‘초두구’, 코로나19 치료 효과 확인",https://n.news.naver.com/mnews/article/366/000...,"권선오 한의학연 책임연구원 연구팀 한의학에서 가슴이나 배 통증, 구토와 같은 증상을...","[예전엔 지식인들만 글을 썼는데, 이젠 하층민들이 자신이 지식인인 줄 알고 글을 쓴..."
61,활기찾은 대학가…코로나19 방역 완화에 매출 ‘껑충’,https://n.news.naver.com/mnews/article/119/000...,신종 코로나바이러스 감염증(코로나19) 방역 완화로 대학교 주변 상권 매출이 크게 ...,[]
